# Section 5.1 Basic Decorators

In [2]:
# Decorator for non-method functions
def pythonic_decorator(func):
    def wrapper(*args,**kwargs):
        print('Now Calling: ' + func.__name__)
        return func(*args,**kwargs)
    return wrapper

@pythonic_decorator
def addNums(a,b):
    return a+b

addNums(7,6)

Now Calling: addNums


13

In [51]:
class Invoice:
    def __init__(self,id_number,price):
        self.id_number = id_number
        self.price = price
        self.owed = price
    def record_payment(self,amount):
        self.owed -= amount

purchase = Invoice(22133,250)
print(purchase.owed)
purchase.record_payment(115.50)
print(purchase.owed)

250
134.5


In [53]:
# Decorator for method functions
def printEnhancedRecord(func):
    def wrapper(self,*args,**kwargs):
        print('Calling {} with id of {}'.format(func.__name__,id(self)))
        return func(self,*args,**kwargs)
    return wrapper

class Invoice:
    def __init__(self,id_number,price):
        self.id_number = id_number
        self.price = price
        self.owed = price
    @printEnhancedRecord
    def record_payment(self,amount):
        self.owed -= amount

    
purchase = Invoice(22133,250)
print(purchase.owed)
purchase.record_payment(115.50)
print(purchase.owed)

250
Calling record_payment with id of 2354168028624
134.5


In [54]:
# Unrelated Python Challenge
# Print Christmas Tree
print('   *') # 3 spaces + 1xs
print('  xxx') # 2 spaces + 3xs
print(' xxxxx') # 1 space + 5xs
print('xxxxxxx') # 0 space + 7xs

   *
  xxx
 xxxxx
xxxxxxx


In [49]:
start = 1
rows = 20
step = 2
goal = rows * step
for i in range(start,goal,step):
    numSpaces = int((goal-i)/step)
    if i ==1:
        char = '*'
    else:
        char = 'x'
    print((' '*numSpaces) + (char*i))


                   *
                  xxx
                 xxxxx
                xxxxxxx
               xxxxxxxxx
              xxxxxxxxxxx
             xxxxxxxxxxxxx
            xxxxxxxxxxxxxxx
           xxxxxxxxxxxxxxxxx
          xxxxxxxxxxxxxxxxxxx
         xxxxxxxxxxxxxxxxxxxxx
        xxxxxxxxxxxxxxxxxxxxxxx
       xxxxxxxxxxxxxxxxxxxxxxxxx
      xxxxxxxxxxxxxxxxxxxxxxxxxxx
     xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
   xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


# Section 5.2 Data in Decorators

In [57]:
def running_averages(func):
    data = {"total":0,"count":0}
    def wrapper(*args,**kwargs):
        value = func(*args,**kwargs)
        data['total'] += value
        data['count'] += 1
        print('so far the avg of {} is {:.01f}'.format(func.__name__,data['total']/data['count']))
        return func(*args,**kwargs)
    return wrapper

In [59]:
@running_averages
def foo(x):
    return x + 3

print(foo(17))
print(foo(71))
print(foo(77))
print(foo(.7))


so far the avg of foo is 20.0
20
so far the avg of foo is 47.0
74
so far the avg of foo is 58.0
80
so far the avg of foo is 44.4
3.7


In [64]:
def collectstats(func):
    data = {"total":0,"count":0}
    def wrapper(*args,**kwargs):
        value = func(*args,**kwargs)
        data['total'] += value
        data['count'] += 1
        return value 
    wrapper.data = data
    return wrapper

In [70]:
@collectstats
def bar(x):
    return x**2

print(bar.data)
bar(4)
bar.data

{'total': 0, 'count': 0}


{'total': 16, 'count': 1}

In [73]:
# WRONG WAY
def countcalls(func):
    count=0
    def wrapper(*args,**kwargs):
        count+=1 #count = count + 1
        print(f"# of calls {count}")
        return func(*args,*kwargs)
    return wrapper 

@countcalls
def foo(x):
    return x+4



In [74]:
foo(4) #local variable error

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [75]:
# RIGHT WAY
def countcalls(func):
    count=0
    def wrapper(*args,**kwargs):
        nonlocal count #nonlocal keyword
        count+=1
        print(f"# of calls {count}")
        return func(*args,*kwargs)
    return wrapper 

@countcalls
def foo(x):
    return x+4

In [76]:
print(foo(1))
print(foo(2))

# of calls 1
5
# of calls 2
6


# Section 5.3 Decorators That Take Arguments

In [77]:
def add(increment):
    def decorator(func):
        def wrapper(n):
            return func(n) + increment
        return wrapper
    return decorator

@add(5)
def foo(n):
    return n **2

foo(1)

6

In [78]:
baz = foo
print(id(foo))
print(id(baz)) # same ids. They are pointing to the same object

2354168669472
2354168669472


# Section 5.4 Class Decorators

In [79]:
#Instantiating functions

class Prefixer:
    def __init__(self,prefix):
        self.prefix = prefix
    def __call__(self,message):
        return self.prefix + message

simonsays = Prefixer('Simon says: ')
simonsays('Hello World')

'Simon says: Hello World'

In [81]:
class PrintLog:
    def __init__(self,func):
        self.func = func
    def __call__(self,*args,**kwargs):
        print('Calling {} with ID of {}'.format(self.func.__name__,id(self)))
        return self.func(*args,**kwargs)

@PrintLog
def add(n):
    return n + 2

add(5)

Calling add with ID of 2354161980368


7

In [86]:
# Works with inheritance
import sys 

class ResultAnnouncer:
    stream = sys.stdout
    prefix = 'RESULT'
    def __init__(self,func):
        self.func = func
    def __call__(self,*args,**kwargs):
        value = self.func(*args,**kwargs)
        self.stream.write(f"{self.prefix}: {value}\n")
        return value

class StdErrResultAnnouncer(ResultAnnouncer):
    stream = sys.stderr
    prefix = 'Error'

In [92]:
@ResultAnnouncer
def do_things(x):
    return x + 5

@StdErrResultAnnouncer
def do_stuff(x):
    return x + 5

do_things(6)

RESULT: 11


11

In [93]:
do_stuff(6)

Error: 11


11

In [101]:
# Also works with data
class CountCalls:
    def __init__(self,func):
        self.count = 0
        self.func = func
    def __call__(self,*args,**kwargs):
        print(f"# of calls: {self.count}")
        self.count += 1
        return self.func(*args,**kwargs)

@CountCalls
def double(x):
    return x * 2

display(
    double(8),
    double(16),
    double(-32),
    double.count
)

# of calls: 0
# of calls: 1
# of calls: 2


16

32

-64

3

In [105]:
# Class based decorator that take arguments
class Add:
    def __init__(self,increment): # doesn't take function but parameter this time
        self.increment = increment 
    def __call__(self,func):
        def wrapper(n): # this is the parameter to the function
            value = func(n) + self.increment
            return value
        return wrapper

@Add(5)
def double(x):
    return 2*x

double(3)

11